## SQL!!!!!

How much to you remember?


In [1]:
import sqlite3
import pandas as pd

con = sqlite3.connect('data.sqlite')

Remember the schema.

In [2]:
schema_df = pd.read_sql("""

SELECT *
FROM sqlite_master


""", con)

schema_df

,type,name,tbl_name,rootpage,sql


Let's get a list of all the table names

In [3]:
table_names = list(schema_df[ schema_df['type'] == 'table']['tbl_name'])
table_names

[]

Now, let's step through that list and display one row from each table

In [4]:
for table in table_names:
    qry = """
    SELECT *
    FROM {}
    LIMIT 1
    """.format(table)
    
    print(f'First row for table {table}')
    display(pd.read_sql(qry, con))
    print('\n\n')

### Simple Query

Which country has the most airports? Which one has the least?

In [5]:
simple_qry = """
    SELECT 
        count(name)
    FROM 
        airports
    GROUP BY
        country
    """ 

pd.read_sql(simple_qry, con)
    
    

DatabaseError: Execution failed on sql '
    SELECT 
        count(name)
    FROM 
        airports
    GROUP BY
        country
    ': no such table: airports

### Where?

Which airports are in the `Pacific/Port_Moresby` time zone? What city and country?

In [6]:
pd.read_sql("""
     SELECT
         name, country, timezone
     FROM 
         airports
     WHERE 
         timezone == 'Pacific/Port_Moresby'
""", con)

DatabaseError: Execution failed on sql '
     SELECT
         name, country, timezone
     FROM 
         airports
     WHERE 
         timezone == 'Pacific/Port_Moresby'
': no such table: airports

### Further down the rabbit hole!

Which countries have between 50 and 100 airports?

In [8]:
pd.read_sql("""
    SELECT
        count(name) as 'airport count'
    FROM 
        airports
    GROUP BY
        country
    HAVING
        'airport count' >= 50 AND 'airport count' <= 100

    
        
""", con)

DatabaseError: Execution failed on sql '
    SELECT
        count(name) as 'airport count'
    FROM 
        airports
    GROUP BY
        country
    HAVING
        'airport count' >= 50 AND 'airport count' <= 100

    
        
': no such table: airports

### Joins!

What is the name of the airline has the most routes? The fewest?

In [11]:
qry_join = """
    SELECT a.name, COUNT() as num_routes
    FROM airlines as a
        JOIN routes as r 
            ON a.id = r.airline_id
    GROUP BY
        a.name
    ORDER BY 
        num_routes DESC
    LIMIT 1
"""

pd.read_sql(qry_join, con)

DatabaseError: Execution failed on sql '
    SELECT a.name, COUNT() as num_routes
    FROM airlines as a
        JOIN routes as r 
            ON a.id = r.airline_id
    GROUP BY
        a.name
    ORDER BY 
        num_routes DESC
    LIMIT 1
': no such table: airlines

In [13]:
sub_qry = """
    SELECT COUNT() AS num_routes
    FROM routes
    GROUP BY airline
    ORDER BY COUNT() DESC
"""

### Subqueries/CTEs!

What is the average number of routes per airline?

In [16]:
qry = """
    WITH number AS (sub_query)
    
    SELECT AVG(num_routes)
    FROM number
"""

pd.read_sql(qry, con)

DatabaseError: Execution failed on sql '
    WITH number AS (sub_query)
    
    SELECT AVG(num_routes)
    FROM number
': near "sub_query": syntax error

### Multiple Joins!

What is the name of the airline with the most departing flights in the `Pacific/Port_Moresby` timezone?

In [17]:
qry = """
    SELECT r.airline, COUNT() num_flights, a.timezone
    FROM routes as r
        INNER JOIN airports as a
            ON r.source_id = a.id
    GROUP BY r.airline, a.timzone
        HAVING a.timezone = 'Pacific/Port_Moresby'
    ORDER BY num_flights DESC
"""

pd.read_sql(qry, con)

DatabaseError: Execution failed on sql '
    SELECT r.airline, COUNT() num_flights, a.timezone
    FROM routes as r
        INNER JOIN airports as a
            ON r.source_id = a.id
    GROUP BY r.airline, a.timzone
        HAVING a.timezone = 'Pacific/Port_Moresby'
    ORDER BY num_flights DESC
': no such table: routes